In [88]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')
import os
import re
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [89]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None)

## Creating a merged DataFrame

In [90]:
files_dir = '/Users/pavelbryantsev/Desktop/Python/KYRREX/Projects/Tokens By Category/html_files'
df = pd.DataFrame()

for file in os.listdir(files_dir):

    #iterate through all files in the directory that end with .html
    if not file.endswith('.html'):
        continue

    with open(f'{files_dir}/{file}', 'r') as f:
        soup = BeautifulSoup(f, 'html.parser')
        table = soup.find_all('table')[0]
        data = pd.read_html(str(table))[0]
        data = data.iloc[1:]
        df = pd.concat([df, data], axis=0)

def extract_string(input_str):
    pattern = r'(\d+)([A-Za-z]+)(\d+)([A-Za-z]+)'
    match = re.search(pattern, input_str)
    if match:
        return match.group(2), match.group(4)
    else:
        return None, None
    
df['Ticker'] = df['↑#Asset'].apply(lambda x: extract_string(x)[0])
df['Name'] = df['↑#Asset'].apply(lambda x: extract_string(x)[1])

In [91]:
df.reset_index(drop=True, inplace=True)

In [92]:
data = df.copy()

## Cleaning the data

In [93]:
def split_string(input_str):
    for idx, item in enumerate(input_str):
        if item == ',':
            return input_str[:idx+6]

In [94]:
# function to check if the string is a number then convert it to float else nan

def convert_to_float(input_str):
    try:
        input_str = input_str.replace('$', '').replace(',', '')

        if input_str.endswith('T'):
            input_str = input_str.replace('T', '')
            return float(input_str[:-1]) * 10**12
        elif input_str.endswith('B'):
            input_str = input_str.replace('$', '')
            return float(input_str[:-1]) * 10**9
        elif input_str.endswith('M'):
            input_str = input_str.replace('$', '')
            return float(input_str[:-1]) * 10**6
        else:
            return float(input_str)
        
    except:
        return np.nan

In [95]:
def clean_columns(df):
    df.columns = df.columns.str.replace('↑', '')
    df.columns = df.columns.str.replace('#', '')
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.replace('Asset', 'Token')
    df['Price'] = df['Price'].apply(convert_to_float)
    df['MarketCap'] = df['MarketCap'].apply(convert_to_float)
    df['24hVolume'] = df['24hVolume'].apply(convert_to_float)
    df['7dVolume'] = df['7dVolume'].apply(convert_to_float)
    df['1mVolume'] = df['1mVolume'].apply(convert_to_float)
    df['FDV'] = df['FDV'].apply(convert_to_float)
    df['FundsRaised'] = df['FundsRaised'].apply(convert_to_float)
    df['Price'] = df['Price'].to_numpy(dtype='float64')
    df['24h%'] = df['24h%'].apply(lambda x: float(x.replace('%', '')))
    df['7d%'] = df['7d%'].apply(lambda x: float(x.replace('%', '')))
    df['1m%'] = df['1m%'].apply(lambda x: float(x.replace('%', '')) if x != '--' else np.nan)
    df['Circ.Supply(%)'] = df['Circ.Supply(%)'].apply(lambda x: float(x.replace('%', '')))
    df['ATH'] = df['ATH'].apply(convert_to_float)
    df['ATHDate'] = pd.to_datetime(df['ATHDate'])
    df['ATLDate'] = pd.to_datetime(df['ATLDate'])
    df['%fromATH'] = df['%fromATH'].apply(lambda x: float(x.replace('%', '')))
    df['%fromATL'] = df['%fromATL'].apply(lambda x: float(x.replace('%', '')))
    df['USDROI'] = df['USDROI'].apply(lambda x: float(x.replace('x', '') if x.endswith('x') else np.nan))
    df['ytd'] = df['ytd'].apply(lambda x: int(x) if x != '--' else 0)
    df['3y'] = df['3y'].apply(lambda x: int(x) if x != '--' else 0)
    df['2y'] = df['2y'].apply(lambda x: int(x) if x != '--' else 0)
    df['1y'] = df['1y'].apply(lambda x: int(x) if x != '--' else 0)
    df['6m'] = df['6m'].apply(lambda x: int(x) if x != '--' else 0)
    df['1m'] = df['1m'].apply(lambda x: int(x) if x != '--' else 0)
    df['ICOPlatform'] = df['ICOPlatform'].apply(lambda x: int(x.replace('+', '')) if isinstance(x, str) and x != '--' and x != np.nan else np.nan)
    df['Investors'] = df['Investors'].apply(lambda x: int(x.replace('+', '')) if isinstance(x, str) and x != '--' and x != np.nan else np.nan)
    df['Exchanges'] = df['Exchanges'].apply(lambda x: int(x.replace('+', '')) if isinstance(x, str) and x != '--' and x != np.nan else np.nan)
    df['TwitterPerformance'] = pd.to_numeric(df['TwitterPerformance'], errors='coerce')
    df.rename(columns={'Circ.Supply(%)': 'CircSupply(%)', 'USDROI': 'USDROI(x)', }, inplace=True) 
    df.drop(columns=['Unnamed:0'], inplace=True)
    df['TradeLaunchDate'] = df['TradeLaunchDate'].apply(lambda x: split_string(x) if x != '--' else np.nan)
    df['TradeLaunchDate'] = pd.to_datetime(df['TradeLaunchDate'], errors='coerce')

    df['days_from_TGE'] = (pd.to_datetime('today') - df['TradeLaunchDate']).dt.days

    df = df[['Name', 'Ticker', 'Price', '24h%', '7d%', '1m%', 'MarketCap', 'FDV', '24hVolume',
             'CircSupply(%)', 'FundsRaised', 'Category', 'ytd', '3y', '2y', '1y',
             '6m', '1m', '7dVolume', '1mVolume', 'ATH', 'ATHDate', '%fromATH',
             'ATLDate', '%fromATL', 'ICOPlatform', 'Investors', 'USDROI(x)',
             'UnlockProgress', 'NextUnlock', 'NextUnlockDate', 'Exchanges',
             'TwitterFollowers', 'TwitterPerformance', 'BullishPeriod',
             'AccumulationPhase', 'TradeLaunchDate', 'days_from_TGE', 'CertikScore', 'Token']]
    
    # drop duplicates
    df.drop_duplicates(subset='Name', inplace=True)
    return df

In [96]:
data = clean_columns(data)

In [97]:
cols = ['FDV', 'MarketCap', 'FundsRaised', '24hVolume', '7dVolume', '1mVolume']
data[cols] = data[cols] / 10**9
data.columns = data.columns.str.replace('MarketCap', 'MarketCap(Bln)')

In [98]:
data = data.sort_values(by='MarketCap(Bln)', ascending=False).reset_index(drop=True)

### Grouping by category

In [99]:
cat_exploring = data.loc[data['Category'] != '--'].groupby('Category').agg({'Category': 'count',
                              'Price': 'mean', 
                              'MarketCap(Bln)': 'sum', 
                              'FDV': 'sum', 
                              '24hVolume': 'sum', 
                              '7dVolume': 'sum', 
                              '1mVolume': 'sum', 
                              'FundsRaised': 'sum', 
                              'ICOPlatform': 'mean', 
                              'Investors': 'mean', 
                              'USDROI(x)': 'median', 
                              'Exchanges': 'mean',  
                              'TwitterPerformance': 'median', 
                              }).sort_values(by='MarketCap(Bln)', ascending=False)
cat_exploring

,Category,Price,MarketCap(Bln),FDV,24hVolume,7dVolume,1mVolume,FundsRaised,ICOPlatform,Investors,USDROI(x),Exchanges,TwitterPerformance
Category,,,,,,,,,,,,,
Blockchain,96,66.803,230.872,311.955,6.726,45.706,162.733,9.157,2.000,12.237,2.270,22.817,496.000
Stablecoin,36,0.900,136.485,138.446,52.992,395.606,1289.625,0.040,NaN,6.500,NaN,18.667,430.000
Cryptocurrency,53,1.913,52.141,78.468,0.896,6.698,23.060,0.390,3.000,10.800,0.210,9.111,143.000
DeFi,267,119.175,46.587,56.976,0.775,5.544,18.436,1.194,1.000,7.024,0.520,9.928,413.000
CEX,19,4.203,21.883,30.281,0.058,0.419,1.688,1.398,1.000,6.571,3.820,11.200,476.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yearn Ecosystem,1,0.008,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,1.000,NaN
Aptos Ecosystem,1,0.001,0.000,0.001,0.000,0.000,0.001,0.000,NaN,NaN,NaN,NaN,131.000
NFT Index,1,0.544,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,34.000


## Lets have a look how the ranking of different sectors has changed over the year 

Will filter tokens that have capitalization over 30 million to minimize shitcoins outlaiers 

In [100]:
rank_change = data.loc[(data['MarketCap(Bln)'] > 0.030) & (data['Category'] != '--'), ['Category', 'ytd', 'MarketCap(Bln)']].sort_values(by='MarketCap(Bln)', ascending=False)
rank_change = rank_change.groupby('Category')['ytd'].describe(percentiles=[.10, .20, .30, .40, .50, .60, .70, .80, .90]).sort_values(by='50%', ascending=False)
rank_change_top = rank_change[(rank_change['count'] > 6) & (rank_change.index != '--')]
rank_change_top.style.background_gradient(axis=None, cmap='bone', subset=['10%', '20%', '30%', '40%', '50%', '60%',	'70%', '80%']).format("{:.1f}")

,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,max
Category,,,,,,,,,,,,,,
Artificial Intelligence (AI),7.0,175.6,321.2,-39.0,11.4,46.0,49.0,51.6,54.0,66.6,90.2,135.8,447.8,893.0
DEX,13.0,598.2,1912.1,-115.0,-33.0,-14.2,-4.0,14.4,28.0,34.6,58.6,137.2,551.2,6932.0
Infrastructure,11.0,50.0,76.5,-46.0,-38.0,-4.0,9.0,11.0,26.0,52.0,84.0,113.0,162.0,181.0
Blockchain Service,12.0,399.8,1175.8,-52.0,-23.9,-11.6,-2.0,1.6,19.5,62.6,120.1,171.2,321.3,4117.0
Cryptocurrency,8.0,872.2,2220.4,-117.0,-35.1,0.0,1.1,8.8,12.5,20.8,44.6,432.6,2382.2,6333.0
Gaming,39.0,411.7,1644.1,-242.0,-57.0,-39.4,-30.0,-7.0,5.0,39.4,82.2,98.8,337.0,8710.0
NFT,20.0,23.6,113.4,-242.0,-43.8,-33.4,-15.7,-4.2,3.0,14.6,30.4,59.2,150.7,304.0
Stablecoin,16.0,632.0,2553.6,-366.0,-77.0,-41.0,-27.0,-4.0,0.0,0.0,2.5,88.0,221.5,10195.0
Blockchain,64.0,223.3,1005.1,-111.0,-34.1,-18.0,-14.1,-8.8,-2.5,-0.2,1.0,34.8,79.3,6160.0


Looks like the hype over AI and Big data influenced capitalization of the tokens that AI related 

In [101]:
px.defaults.template = 'plotly_dark'

fig = px.bar(rank_change_top, 
             y=rank_change_top.index, 
             x='50%', 
             color='50%', 
             title='Median Rank Change by Category', 
             labels={'50%': 'Rank Change', 'index': 'Category'})

# add annotations to the bars
for i in range(len(rank_change_top)):
    fig.add_annotation(x=rank_change_top['50%'][i], y=rank_change_top.index[i], 
                       text=f'{rank_change_top["50%"][i]:.0f}', 
                       showarrow=False, 
                       font=dict(size=11, color='white'),
                       xshift=11)
# remove x axis labels

fig.update_xaxes(showgrid=False, showticklabels=False, title='')
fig.update_yaxes(title='')
fig.update_layout(coloraxis_showscale=False,
                  yaxis={'categoryorder':'total ascending'},
                  autosize=False, width=800, height=400,
                  margin=dict(l=0, r=50, t=50, b=0))

fig.show()

## Exploring new tokens 

Lets take a look to tokens that are younger then one year 

In [102]:
new_tokens = data.loc[(data['days_from_TGE'] < 365) & (data['Category'] != '--'), ['Category', 'days_from_TGE', 'MarketCap(Bln)']].sort_values(by='MarketCap(Bln)', ascending=False)
new_tokens = new_tokens.groupby('Category').agg({'MarketCap(Bln)': ['count', 'sum']}).sort_values(by=('MarketCap(Bln)', 'sum'), ascending=False)
new_tokens_top = new_tokens[(new_tokens[('MarketCap(Bln)', 'count')] > 6)]
new_tokens_top.style.background_gradient(axis=0, low=0.37, cmap='bone').format("{:.3f}")

In [103]:
fig = px.bar(new_tokens_top,
                y=new_tokens_top.index,
                x=new_tokens_top['MarketCap(Bln)']['sum'].values,
                color=new_tokens_top['MarketCap(Bln)']['sum'].values,
                color_continuous_scale='Bluered_r',
                title='Market Cap Tokens Launched in the Last Year by Category',
                labels={'y': 'Category', 'x': 'Market Cap (Bln)'},
                text_auto=True)
    
fig.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False,
                  hovertemplate='%{x:.3f} B', 
                #   marker=dict(line=dict(color='white', width=0.5)),
                  texttemplate='%{x:.3f} B')

fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, title='')
fig.update_yaxes(title='')
fig.update_layout(coloraxis_showscale=False,
                  yaxis={'categoryorder':'total ascending'},
                  autosize=False, width=800, height=500,
                  margin=dict(l=0, r=50, t=50, b=0))
fig.show()

In [104]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(
    x=new_tokens_top.index,
    y=new_tokens_top['MarketCap(Bln)']['sum'].values),   
    # name='Primary Product',

    secondary_y=False
)
fig.add_trace(go.Bar(
    x=new_tokens_top.index,
    y=new_tokens_top['MarketCap(Bln)']['count'].values),
    # name='Secondary Product',

    secondary_y=True
)
# add secondary y-axis for 'count'
fig.update_layout(
    title='Market Cap Tokens Launched in the Last Year by Category',
    xaxis_tickangle=-45,
    xaxis_title='Category',
    yaxis_title='Market Cap (Bln)',
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1, # gap between bars of the same location coordinates.
    autosize=False, width=800, height=500,
    margin=dict(l=80, r=50, t=50, b=0)
)
fig.layout.template = 'plotly_dark'
# fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

## FDV exploting by category

Lets also create a few new ratios

In [105]:
data['FDV/MarketCap'] = data['FDV'] / data['MarketCap(Bln)']
data['Raised/Cap'] = data['FundsRaised'] / data['MarketCap(Bln)']

In [106]:
fdv_tab = data.loc[data['Category'] != '--'].groupby('Category')['FDV/MarketCap'].describe().sort_values(by='50%', ascending=False)
fdv_tab = fdv_tab[(fdv_tab['count'] > 6)]
fdv_tab.style.background_gradient(axis=0, low=0.37, cmap='bone').format("{:.2f}")

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
Web 3.0,17.00,18.82,30.87,1.00,1.72,6.22,16.07,100.00
BSCPad IDO,9.00,7.87,9.07,1.00,1.54,5.17,9.23,30.03
Avalanche Ecosystem,10.00,11.71,17.88,1.00,2.51,5.17,7.13,57.70
Arbitrum Ecosystem,10.00,14.09,30.41,1.00,2.18,4.00,6.25,99.94
Solana Ecosystem,18.00,30.66,73.84,1.00,1.55,3.76,11.33,245.76
AMM,9.00,3.90,3.25,1.00,1.19,3.57,5.37,10.27
Entertainment,8.00,5.12,4.48,1.00,2.44,3.55,6.36,14.16
Ignition IDO,9.00,4.49,3.68,1.00,1.94,3.29,5.40,11.28
Sport,35.00,4.86,7.24,1.00,1.68,3.06,4.61,42.68


## Tokens that drops from ATH the most 

In [107]:
drop_cat = data.loc[(data['Category'] != '--') & (data['MarketCap(Bln)'] > 0.030)]
drop_cat = drop_cat.groupby('Category')['%fromATH'].describe().sort_values(by='50%', ascending=False)
drop_cat = drop_cat[(drop_cat['count'] > 3)]
drop_cat_gradient = drop_cat.style.background_gradient(axis=0, low=0.37, cmap='bone').format("{:.2f}")
drop_cat_gradient

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
Artificial Intelligence (AI),7.00,-28.77,35.22,-94.63,-42.48,-8.57,-5.86,-1.53
Stablecoin,16.00,-26.89,22.11,-80.88,-40.79,-19.32,-12.32,-2.34
Backed Tokens,5.00,-50.88,29.95,-97.82,-62.26,-39.79,-30.15,-24.39
DEX,13.00,-49.47,38.10,-97.82,-93.47,-42.89,-12.33,-10.74
Layer-2,6.00,-55.77,43.26,-99.79,-94.40,-57.66,-16.88,-9.60
CEX,12.00,-65.69,26.89,-98.41,-90.62,-66.84,-50.10,-18.39
Big Data,4.00,-64.29,36.07,-98.55,-88.15,-70.95,-47.09,-16.72
Meme,17.00,-68.38,27.89,-100.00,-87.65,-77.83,-54.18,-5.87
Blockchain Service,12.00,-67.96,28.48,-96.71,-84.30,-78.28,-60.52,-4.70


### Average age of the token in each category.

In [108]:
avg_age_cat = data.loc[data['Category'] != '--'].groupby('Category')['days_from_TGE'].describe().sort_values(by='mean', ascending=False)
avg_age_cat_gradient = avg_age_cat[avg_age_cat['count'] > 6].style.background_gradient(axis=0, low=0.37, cmap='bone').format("{:.0f}")
avg_age_cat_gradient

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
Nodes,23,2197,607,549,1856,2237,2454,3530
Privacy Coin,34,2136,752,980,1581,1996,2516,3608
IOT,7,1892,531,1117,1534,1907,2354,2449
Storage,7,1735,742,189,1700,1902,2106,2446
Media,24,1688,585,378,1272,1712,2171,2702
Finance,28,1592,478,762,1312,1539,1833,2482
Big Data,14,1470,591,223,1048,1534,1997,2239
Cosmos Ecosystem,11,1453,524,669,1114,1346,1864,2393
Blockchain,96,1436,884,55,765,1326,2120,3672


In [109]:
from IPython.display import HTML

def side_by_side(*dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

In [110]:
side_by_side(avg_age_cat_gradient, drop_cat_gradient)

,count,mean,std,min,25%,50%,75%,max
Category,,,,,,,,
Nodes,23,2197,607,549,1856,2237,2454,3530
Privacy Coin,34,2136,752,980,1581,1996,2516,3608
IOT,7,1892,531,1117,1534,1907,2354,2449
Storage,7,1735,742,189,1700,1902,2106,2446
Media,24,1688,585,378,1272,1712,2171,2702
Finance,28,1592,478,762,1312,1539,1833,2482
Big Data,14,1470,591,223,1048,1534,1997,2239
Cosmos Ecosystem,11,1453,524,669,1114,1346,1864,2393
Blockchain,96,1436,884,55,765,1326,2120,3672


In [145]:
fig = px.bar(avg_age_cat[avg_age_cat['count'] > 6].sort_values(by='mean', ascending=False),
             x='mean', 
             y=avg_age_cat[avg_age_cat['count'] > 6].index, 
             color='mean', 
             title='Mean Age of Tokens by Category', 
             labels={'Mean': 'Mean', 'index': 'Category'},
             text_auto=True)


fig.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False,
                  hovertemplate='%{x:.0f} days', 
                  texttemplate='%{x:.0f} days')
fig.update_xaxes(showgrid=False, showticklabels=False, title='')
# fig.update_yaxes(categoryorder='total ascending')
# fig.update_xaxes(autorange='reversed')
fig.update_layout(autosize=False, width=800, height=600, 
                  margin=dict(l=0, r=50, t=50, b=0),
                  coloraxis_showscale=False)
fig.show()

In [139]:
fig = px.bar(drop_cat, 
             x='mean', 
             y=drop_cat.index, 
             color='mean', 
             title='Mean % from ATH by Category', 
             labels={'Mean': '% from ATH', 'index': 'Category'})

for i in range(len(drop_cat)):
    fig.add_annotation(x=drop_cat['mean'][i], y=drop_cat.index[i], 
                       text=f'{drop_cat["mean"][i]:.2f}', 
                       showarrow=False, 
                       font=dict(size=11, color='white'),
                       xshift=14)

fig.update_xaxes(showgrid=False, showticklabels=False, title='')
fig.update_yaxes(categoryorder='total ascending')
fig.update_xaxes(autorange='reversed')
fig.update_layout(autosize=False, width=800, height=500, 
                  margin=dict(l=0, r=50, t=50, b=0),
                  coloraxis_showscale=False)
fig.show()

### Number of new tokens by year 

In [112]:
data['year'] = pd.to_datetime(data['TradeLaunchDate']).dt.year

In [113]:
piv_table = data[data['Category'] != '--'].pivot_table(index='year', 
                                           columns='Category', 
                                           values='Token', 
                                           aggfunc='count').fillna(0)

piv_table = piv_table.loc[:, (piv_table > 6).any()]
piv_table.T.style.background_gradient(axis=1, cmap='bone').format("{:.0f}")

year,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Category,,,,,,,,,,,,
Artificial Intelligence (AI),0,0,0,0,1,1,1,0,2,0,16,1
Avalanche Ecosystem,0,0,0,0,1,0,0,0,7,1,0,1
BNB Chain Ecosystem,0,0,0,0,0,1,1,4,12,5,6,2
BSCPad IDO,0,0,0,0,0,0,0,0,9,0,0,0
Backed Tokens,0,0,0,0,0,0,3,7,1,0,0,0
Bitcoin Ecosystem,0,0,0,0,0,0,0,0,0,0,12,1
Blockchain,0,4,0,4,11,10,16,11,15,12,12,1
Blockchain Service,0,0,0,0,0,2,1,4,3,2,7,1
CEX,0,0,0,0,2,1,7,1,5,2,1,0


### Total raised by cat

In [195]:
raised_cat = data.groupby('Category')['FundsRaised'].agg(['count', 'sum', 'mean', 'median']).sort_values(by='sum', ascending=False)
raised_cat = raised_cat[raised_cat['count'] > 10]
raised_cat[raised_cat.index != '--'].style.background_gradient(axis=0, cmap='bone').format("{:.3f}")

,count,sum,mean,median
Category,,,,
Blockchain,46.000,9.157,0.199,0.056
CEX,12.000,1.398,0.116,0.036
NFT,56.000,1.232,0.022,0.004
DeFi,109.000,1.194,0.011,0.003
Gaming,81.000,1.078,0.013,0.003
Blockchain Service,14.000,0.477,0.034,0.017
Media,13.000,0.458,0.035,0.027
DEX,23.000,0.426,0.019,0.005
Finance,11.000,0.413,0.038,0.030


In [194]:
fig = px.bar(raised_cat[raised_cat.index != '--'],
                y=raised_cat[raised_cat.index != '--'].index,
                x='sum',
                color='sum',
                title='Total Funds Raised by Category',
                labels={'sum': 'Total Funds Raised', 'index': 'Category'},
                text_auto=True)

fig.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False,
                #   hovertemplate='%{y:.3f} B',)
                  texttemplate='%{x:.3f} B')
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=True)
fig.update_xaxes(title='')
fig.update_layout(coloraxis_showscale=False,
                  yaxis={'categoryorder':'total ascending'},
                  autosize=False, width=800, height=500,
                  margin=dict(l=0, r=50, t=50, b=0))
fig.show()